In [14]:
import pandas as pd
import geopandas as gpd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import itertools
import sys
sys.path.append('..')
from security_game.target import Target
from security_game.green_security_game import GreenSecurityGame
from security_game.infra_security_game import InfraSecurityGame

from solvers.mip import mip
from solvers.nash import nash
from solvers.double_oracle import double_oracle
from solvers.double_oracle_sf import double_oracle_sf
from solvers.no_regret import regret_matching

from utils.random_utils import generate_random_utility_matrix_like, generate_random_target_utility_matrix_like, generate_random_schedule_mapping_like

import time
from matplotlib import pyplot as plt
from collections import defaultdict, Counter

In [15]:
seeds = [1,2,3,4,5,6,7,8,9,10]

# GSG

## NFG

In [16]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)

In [17]:
gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=1, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=9, 
             generate_utility_matrix=True, 
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=False, 
             general_sum=False, 
             **schedule_form_kwargs)

In [18]:
gsg.utility_matrix.shape

(41479, 11)

In [19]:
nruntimes = []
nsupports = []
nus = []
mip_us_per_seed = []
mip_supports_per_seed = []
mip_runtimes_per_seed = []

for seed in seeds:
    print(f"starting seed {seed}")
    random_matrix = generate_random_utility_matrix_like(gsg.utility_matrix)
    start = time.time()
    nD_a, nD_d, nu = nash(random_matrix)
    end = time.time()
    nruntime = end-start
    nsupport = sum([1 for p in nD_d if p!=0])
    nus.append(nu)
    print(nsupport,nu)
    nruntimes.append(nruntime)
    nsupports.append(nsupport)
    
    mip_us = []
    mip_supports = []
    mip_runtimes = []
    
    for i in range(1,60):
        start = time.time()
        print(f"starting i={i} at time {start}")
        mu, mD_d = mip(random_matrix,i)
        end = time.time()
        print(f"finished i={i} in {end-start} seconds with u={mu}")
        msupport = sum([1 for p in mD_d if p!=0])
        mip_us.append(mu)
        mip_supports.append(i)
        mip_runtimes.append(end-start)
        if abs(mu-nu) <= 1e-12:
            mip_us_per_seed.append(mip_us)
            mip_supports_per_seed.append(mip_supports)
            mip_runtimes_per_seed.append(mip_runtimes)
            break

starting seed 1
11 -0.18759242338907556
starting i=1 at time 1746424366.380344
finished i=1 in 32.253100633621216 seconds with u=-0.33049861334842967
starting i=2 at time 1746424398.6355274
finished i=2 in 34.95654082298279 seconds with u=-0.21206488911779517
starting i=3 at time 1746424433.5930698
finished i=3 in 263.79923248291016 seconds with u=-0.2027180992745322
starting i=4 at time 1746424697.3943052
finished i=4 in 89.53590774536133 seconds with u=-0.19611201169804243
starting i=5 at time 1746424786.931218
finished i=5 in 55.00484824180603 seconds with u=-0.19191852234427761
starting i=6 at time 1746424841.9370904
finished i=6 in 61.43380928039551 seconds with u=-0.19014835729000226
starting i=7 at time 1746424903.3719082
finished i=7 in 18.713496685028076 seconds with u=-0.18850623369178512
starting i=8 at time 1746424922.0864074
finished i=8 in 42.845847368240356 seconds with u=-0.18818386343547683
starting i=9 at time 1746424964.9342601
finished i=9 in 11.174825668334961 seco

In [20]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps":[9 for i in range(len(seeds))],
    "num_attackers":[1 for i in range(len(seeds))],
    "num_defenders":[1 for i in range(len(seeds))],
    "num_clusters":[10 for i in range(len(seeds))],
    "dims":[7 for i in range(len(seeds))],
    "defense_time_threshold":[1 for i in range(len(seeds))],
    "force_return":[False for i in range(len(seeds))],
    "num_defender_actions": [len(gsg.defender_actions) for i in range(len(seeds))],
    "nash_value":nus,
    "nash_support":nsupports,
    "nash_runtime":nruntimes,
    "mip_value":mip_us_per_seed,
    "mip_support":mip_supports_per_seed,
    "mip_runtime":mip_runtimes_per_seed,
})
df


,seed,num_timesteps,num_attackers,num_defenders,num_clusters,dims,defense_time_threshold,force_return,num_defender_actions,nash_value,nash_support,nash_runtime,mip_value,mip_support,mip_runtime
0,1,9,1,1,10,7,1,False,41479,-0.187592,11,3.220002,"[-0.33049861334842967, -0.21206488911779517, -...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[32.253100633621216, 34.95654082298279, 263.79..."
1,2,9,1,1,10,7,1,False,41479,-0.198526,11,3.397532,"[-0.4201333011262066, -0.2679993774682173, -0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[25.359724283218384, 757.6126692295074, 120.91..."
2,3,9,1,1,10,7,1,False,41479,-0.191004,11,3.038714,"[-0.3475327050726812, -0.2580880780591269, -0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[30.28805661201477, 4684.116407871246, 245.474..."
3,4,9,1,1,10,7,1,False,41479,-0.187622,11,2.983597,"[-0.38593971907107893, -0.26684074604678143, -...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[56.12270784378052, 878.412425994873, 67.96332..."
4,5,9,1,1,10,7,1,False,41479,-0.176030,10,2.995340,"[-0.2906412128056464, -0.21102580904812476, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[12.671159029006958, 741.9412114620209, 109.17..."
5,6,9,1,1,10,7,1,False,41479,-0.176390,11,3.037144,"[-0.33458244851934105, -0.22352023104467122, -...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[23.87569284439087, 534.5615437030792, 49.8034..."
6,7,9,1,1,10,7,1,False,41479,-0.180118,11,3.040523,"[-0.3367153020684932, -0.22492398547957598, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[23.55077338218689, 1304.2784504890442, 294.91..."
7,8,9,1,1,10,7,1,False,41479,-0.203662,11,3.031541,"[-0.3847404347260328, -0.25670419923314197, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[54.090611934661865, 708.6245937347412, 428.12..."
8,9,9,1,1,10,7,1,False,41479,-0.196674,11,3.044220,"[-0.43179747045488504, -0.27416749950096725, -...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[1393.2623889446259, 2182.9206199645996, 243.2..."
9,10,9,1,1,10,7,1,False,41479,-0.195059,11,3.066049,"[-0.34436445322944637, -0.24031442702318015, -...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[29.169074296951294, 1596.7552001476288, 367.2..."


In [21]:
df.to_csv("GSG_NFG_T9_1D1A_SPARSITY_RANDOM_MATRIX.csv")

### SF

In [3]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post),(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=10, 
             generate_utility_matrix=True, 
             defense_time_threshold=2, 
             generate_actions=False, 
             force_return=False, 
             general_sum=False, 
             **schedule_form_kwargs)

In [4]:
gsg.schedule_form_dict["defender_utility_matrix"].shape

(2401, 10)

In [5]:
start = time.time()
nD_a, nD_d, nu = nash(gsg.schedule_form_dict["defender_utility_matrix"])
end = time.time()
nruntime = end-start
nsupport = sum([1 for p in nD_d if p!=0])
nsupport,nu

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2455389
Academic license 2455389 - for non-commercial use only - registered to jc___@columbia.edu


(8, -0.2892405920467004)

In [6]:
nruntimes = []
nsupports = []
nus = []
mip_us_per_seed = []
mip_supports_per_seed = []
mip_runtimes_per_seed = []

for seed in seeds:
    print(f"starting seed {seed}")
    random_matrix = generate_random_utility_matrix_like(gsg.schedule_form_dict["defender_utility_matrix"])
    print(random_matrix.shape)
    start = time.time()
    nD_a, nD_d, nu = nash(random_matrix)
    end = time.time()
    nruntime = end-start
    nsupport = sum([1 for p in nD_d if p!=0])
    nus.append(nu)
    print(nsupport,nu)
    nruntimes.append(nruntime)
    nsupports.append(nsupport)
    
    mip_us = []
    mip_supports = []
    mip_runtimes = []
    
    for i in range(1,60):
        start = time.time()
        print(f"starting i={i} at time {start}")
        mu, mD_d = mip(random_matrix,i)
        end = time.time()
        print(f"finished i={i} in {end-start} seconds with u={mu}")
        msupport = sum([1 for p in mD_d if p!=0])
        mip_us.append(mu)
        mip_supports.append(i)
        mip_runtimes.append(end-start)
        if abs(mu-nu) <= 1e-12:
            mip_us_per_seed.append(mip_us)
            mip_supports_per_seed.append(mip_supports)
            mip_runtimes_per_seed.append(mip_runtimes)
            break

starting seed 1
(2401, 10)
8 -0.25360478726289115
starting i=1 at time 1746419677.8438423
finished i=1 in 0.7710902690887451 seconds with u=-0.40068249102802195
starting i=2 at time 1746419678.6149325
finished i=2 in 4.629194021224976 seconds with u=-0.3176999753081526
starting i=3 at time 1746419683.2441266
finished i=3 in 2.6587727069854736 seconds with u=-0.28974976960374516
starting i=4 at time 1746419685.9028993
finished i=4 in 1.2794559001922607 seconds with u=-0.27217254489644127
starting i=5 at time 1746419687.1823552
finished i=5 in 1.3438737392425537 seconds with u=-0.26462832555160387
starting i=6 at time 1746419688.526229
finished i=6 in 0.3175954818725586 seconds with u=-0.25544713252285756
starting i=7 at time 1746419688.8448899
finished i=7 in 0.5570852756500244 seconds with u=-0.2540559665539354
starting i=8 at time 1746419689.4029977
finished i=8 in 0.26291918754577637 seconds with u=-0.25360478726289104
starting seed 2
(2401, 10)
9 -0.2631362649722988
starting i=1 at 

In [9]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps":[10 for i in range(len(seeds))],
    "num_attackers":[1 for i in range(len(seeds))],
    "num_defenders":[2 for i in range(len(seeds))],
    "num_clusters":[10 for i in range(len(seeds))],
    "dims":[7 for i in range(len(seeds))],
    "defense_time_threshold":[2 for i in range(len(seeds))],
    "force_return":[False for i in range(len(seeds))],
    "num_defender_actions": [len(gsg.schedule_form_dict["defender_actions"]) for i in range(len(seeds))],
    "nash_value":nus,
    "nash_support":nsupports,
    "nash_runtime":nruntimes,
    "mip_value":mip_us_per_seed,
    "mip_support":mip_supports_per_seed,
    "mip_runtime":mip_runtimes_per_seed,
})
df


,seed,num_timesteps,num_attackers,num_defenders,num_clusters,dims,defense_time_threshold,force_return,num_defender_actions,nash_value,nash_support,nash_runtime,mip_value,mip_support,mip_runtime
0,1,10,1,2,10,7,2,False,2401,-0.253605,8,0.236281,"[-0.40068249102802195, -0.3176999753081526, -0...","[1, 2, 3, 4, 5, 6, 7, 8]","[0.7710902690887451, 4.629194021224976, 2.6587..."
1,2,10,1,2,10,7,2,False,2401,-0.263136,9,0.173865,"[-0.44987933261671675, -0.34632841673516046, -...","[1, 2, 3, 4, 5, 6, 7, 8, 9]","[0.5698845386505127, 10.735763549804688, 5.367..."
2,3,10,1,2,10,7,2,False,2401,-0.270763,10,0.171223,"[-0.4811407822614249, -0.3309055233967805, -0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0.6406888961791992, 3.4138107299804688, 2.337..."
3,4,10,1,2,10,7,2,False,2401,-0.280383,10,0.172195,"[-0.46118024995065876, -0.3426789214769698, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0.6343221664428711, 1.2823545932769775, 5.480..."
4,5,10,1,2,10,7,2,False,2401,-0.248891,10,0.172318,"[-0.4965335556086801, -0.2926772833478436, -0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0.7090640068054199, 0.9008779525756836, 0.853..."
5,6,10,1,2,10,7,2,False,2401,-0.252663,9,0.179286,"[-0.468225517657905, -0.31069253834598015, -0....","[1, 2, 3, 4, 5, 6, 7, 8, 9]","[1.2646329402923584, 2.888512372970581, 3.8674..."
6,7,10,1,2,10,7,2,False,2401,-0.275818,10,0.177366,"[-0.5542275947385978, -0.3617897167056349, -0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1.1114680767059326, 15.565851211547852, 8.417..."
7,8,10,1,2,10,7,2,False,2401,-0.279012,10,0.184612,"[-0.4691986366535408, -0.3380366030038281, -0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0.7301604747772217, 4.078875541687012, 5.4972..."
8,9,10,1,2,10,7,2,False,2401,-0.247238,9,0.244623,"[-0.37689756841129995, -0.30072469786632616, -...","[1, 2, 3, 4, 5, 6, 7, 8, 9]","[2.8923544883728027, 1.7235264778137207, 0.822..."
9,10,10,1,2,10,7,2,False,2401,-0.265956,10,0.210652,"[-0.44210547472802414, -0.3388184199335233, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0.8763759136199951, 4.857979774475098, 2.6047..."


In [10]:
df.to_csv("GSG_SF_T10_SPARSITY_RANDOM_MATRIX.csv")